https://stepik.org/lesson/1576178/step/7

In [ ]:
# @title загрузка датасета Digits на диск
from sklearn.datasets import load_digits
import torch


digits = load_digits()

# Признаки и целевые значения
_global_var_data_x = torch.tensor(digits.data, dtype=torch.float32)
_global_var_target = torch.tensor(digits.target, dtype=torch.int64)

# Размеры и типы
print("Размеры и типы:", _global_var_data_x.shape, _global_var_data_x.dtype, _global_var_target.shape, _global_var_target.dtype)

# Прочие характеристики
print("Количество классов: ", len(digits.target_names))
print("Названия классов: ", digits.target_names)
print("Названия признаков: ", digits.feature_names)

# Первые строки данных
print("Первые 5 строк признаков:\n", _global_var_data_x[:5])
print("Первые 5 целевых значений:\n", _global_var_target[:5])

Размеры и типы: torch.Size([1797, 64]) torch.float32 torch.Size([1797]) torch.int64
Количество классов:  10
Названия классов:  [0 1 2 3 4 5 6 7 8 9]
Названия признаков:  ['pixel_0_0', 'pixel_0_1', 'pixel_0_2', 'pixel_0_3', 'pixel_0_4', 'pixel_0_5', 'pixel_0_6', 'pixel_0_7', 'pixel_1_0', 'pixel_1_1', 'pixel_1_2', 'pixel_1_3', 'pixel_1_4', 'pixel_1_5', 'pixel_1_6', 'pixel_1_7', 'pixel_2_0', 'pixel_2_1', 'pixel_2_2', 'pixel_2_3', 'pixel_2_4', 'pixel_2_5', 'pixel_2_6', 'pixel_2_7', 'pixel_3_0', 'pixel_3_1', 'pixel_3_2', 'pixel_3_3', 'pixel_3_4', 'pixel_3_5', 'pixel_3_6', 'pixel_3_7', 'pixel_4_0', 'pixel_4_1', 'pixel_4_2', 'pixel_4_3', 'pixel_4_4', 'pixel_4_5', 'pixel_4_6', 'pixel_4_7', 'pixel_5_0', 'pixel_5_1', 'pixel_5_2', 'pixel_5_3', 'pixel_5_4', 'pixel_5_5', 'pixel_5_6', 'pixel_5_7', 'pixel_6_0', 'pixel_6_1', 'pixel_6_2', 'pixel_6_3', 'pixel_6_4', 'pixel_6_5', 'pixel_6_6', 'pixel_6_7', 'pixel_7_0', 'pixel_7_1', 'pixel_7_2', 'pixel_7_3', 'pixel_7_4', 'pixel_7_5', 'pixel_7_6', 'pixel_7_7

In [ ]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim


class DigitsDataset(data.Dataset):
    def __init__(self):
        self.data = _global_var_data_x   # тензор размерностью (178, 13), тип float32
        self.target = _global_var_target # тензор размерностью (178, ), тип int64 (long)

        self.length = len(self.target)
        self.categories = ['class_0' 'class_1' 'class_2'] # названия классов

    def __getitem__(self, item):
        return self.data[item], self.target[item],

    def __len__(self):
        return self.length

class DigitsClassModel(nn.Module):
    def __init__(self, in_features=64, out_features=10):
        super().__init__()
        self.layer1 = nn.Linear(in_features, 32)
        self.layer2 = nn.Linear(32, 16)
        self.layer3 = nn.Linear(16, out_features)

    def forward(self, x):
        x = self.layer1(x).relu()
        x = self.layer2(x).relu()
        x = self.layer3(x)
        return x

# Создание модели DigitsClassModel и перевод в режим обучения
model = DigitsClassModel(in_features=64, out_features=10)
model.train()

# Параметры градиентного спуска
epochs = 10     # число эпох обучения
batch_size = 16 # размер батча
lr = 0.01

# Создание объекта класса DigitsDataset с разбиением на батчи
d_train = DigitsDataset()
train_data = data.DataLoader(d_train, batch_size, shuffle=True, drop_last=False)

# Оптимизатор и функция потерь
optimizer = optim.Adam(params=model.parameters(), lr=lr)
loss_func = torch.nn.CrossEntropyLoss()

# Градиентный спуск
for _e in range(epochs): # итерации по эпохам
    for x_train, y_train in train_data:
        predict = model(x_train).squeeze()
        loss = loss_func(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Перевод модели в режим эксплуатации
model.eval()

# Прогноз модели по всем данным выборки
y_pred = torch.argmax(model(d_train.data), dim=1)

# Качество
Q = (y_pred.data == d_train.target).sum().item() / len(d_train)
# print(Q)

0.9604897050639956
